# Algorytmy macierzowe
## lab 2

In [1]:
import numpy as np
from time import time
from matplotlib import pyplot as plt
import pandas as pd

Własna klasa Number do zliczania ilości operacji

In [2]:
class Number(float):
    operation_counter = 0
    
    def __repr__(self) -> str:
        return f"{self:.8f}"
    
    def __radd__(self, other):
        Number.operation_counter += 1
        return Number(super().__radd__(other))

    def __add__(self, other):
        Number.operation_counter += 1
        return Number(super().__add__(other))
    
    def __rsub__(self, other):
        Number.operation_counter += 1
        return Number(super().__rsub__(other))

    def __sub__(self, other):
        Number.operation_counter += 1
        return Number(super().__sub__(other))
    
    def __mul__ (self, other):
        Number.operation_counter += 1
        return Number(super().__mul__(other))
    
    def __rmul__ (self, other):
        Number.operation_counter += 1
        return Number(super().__rmul__(other))
    
    def __truediv__(self, other):
        Number.operation_counter += 1
        return Number(super().__truediv__(other))
    
    def __rtruediv__(self, other):
        Number.operation_counter += 1
        return Number(super().__rtruediv__(other))
    
    def counter_reset():
        Number.operation_counter = 0

Generowanie macierzy

In [3]:
def random_matrix(matrix_size, min_val, max_val):
    """Return matrix with random Number floats from [min_val, max_val)"""
    matrix = (max_val - min_val) * np.random.random(matrix_size) + min_val
    return np.array([[Number(value) for value in row] for row in matrix], dtype=Number)

def transform_to_float(A):
    """np.linalg.inv doesnt work with dtype=Number"""
    return np.array(A, dtype=float)

In [4]:
exp = 2
matrix_size = (2**exp, 2**exp)

min_val = 0.00000001
max_val = 1

A = random_matrix(matrix_size, min_val, max_val)

### Rekurencyjne odwracanie macierzy

In [5]:
def inverse(A):
    if A.shape[0] != A.shape[1]:
        print("ERROR: Wrong matrix size!")
        return None
    
    if len(A) == 1:
        return np.array([[1/A[0,0]]])
    
    if len(A) == 2:
        return np.array(
            [[A[1,1], -A[0,1]],
             [-A[1, 0], A[0,0]]]
        ) / (A[0,0]*A[1,1] - A[0,1]*A[1,0])
        
    matrix_size = len(A)
    A11 = A[:matrix_size//2, :matrix_size//2]
    A12 = A[:matrix_size//2, matrix_size//2:]
    A21 = A[matrix_size//2:, :matrix_size//2]
    A22 = A[matrix_size//2:, matrix_size//2:]
    
    A11_inv = inverse(A11)
    S22 = A22 - A21 @ A11_inv @ A12
    S22_inv = inverse(S22)
    B11 = A11_inv @ (np.identity(len(A11)) + A12 @ S22_inv @ A21 @ A11_inv)
    B12 = -A11_inv @ A12 @ S22_inv
    B21 = -S22_inv @ A21 @ A11_inv
    B22 = S22_inv
    
    return np.vstack((np.hstack((B11, B12)), np.hstack((B21, B22))))

In [6]:
Number.counter_reset()
res = inverse(A)

print(f"Matrix size: {matrix_size}")
print("Number of operations:", Number.operation_counter)
print("Is correct?:", np.allclose(transform_to_float(res), np.linalg.inv(transform_to_float(A))))

Matrix size: (4, 4)
Number of operations: 134
Is correct?: True


### Rekurencyjna LU faktoryzacja

In [7]:
def LU_factorization(A):
    if A.shape[0] != A.shape[1]:
        print("ERROR: Wrong matrix size!")
        return None
    
    if len(A) == 1:
        L = np.array([[Number(1.0)]], dtype=Number)
        U = A.copy()
        return L, U
    
    matrix_size = len(A)
    A11 = A[:matrix_size//2, :matrix_size//2]
    A12 = A[:matrix_size//2, matrix_size//2:]
    A21 = A[matrix_size//2:, :matrix_size//2]
    A22 = A[matrix_size//2:, matrix_size//2:]
    
    L11, U11 = LU_factorization(A11)
    U11_inv = inverse(U11)
    L21 = A21 @ U11_inv
    L11_inv = inverse(L11)
    U12 = L11_inv @ A12
    S = A22 - A21 @ U11_inv @ L11_inv @ A12
    LS, US = LU_factorization(S)
    
    L = np.vstack((np.hstack((L11, np.zeros(A11.shape))), np.hstack((L21, LS))))
    U = np.vstack((np.hstack((U11, U12)), np.hstack((np.zeros(A11.shape), US))))
    return L, U

In [8]:
Number.counter_reset()
L, U = LU_factorization(A)

print(f"Matrix size: {matrix_size}")
print("Number of operations:", Number.operation_counter)
print("Is correct?:", np.allclose(transform_to_float(A), transform_to_float(L @ U)))


Matrix size: (4, 4)
Number of operations: 86
Is correct?: True


### Rekurencyjne obliczanie wyznacznika

In [9]:
def det(A):
    L, U = LU_factorization(A)
    result = 1
    for i in range(len(A)):
        result *= L[i, i] * U[i, i]
    return result

In [10]:
Number.counter_reset()
A_det = det(A)

print(f"Matrix size: {matrix_size}")
print("Number of operations:", Number.operation_counter)
print("Is correct?:", abs(A_det - np.linalg.det(transform_to_float(A))) < 1e-8)

Matrix size: (4, 4)
Number of operations: 94
Is correct?: True


### Testy

In [11]:
sizes = [2 ** e for e in range(1, 8)]

In [12]:
res_times = []
res_operations = []

for n in sizes:
    A = random_matrix((n, n), min_val, max_val)
    
    Number.counter_reset()
    t_start = time()
    inverse(A)
    inverse_time = time() - t_start
    inverse_op = Number.operation_counter
    
    Number.counter_reset()
    t_start = time()
    LU_factorization(A)
    lu_time = time() - t_start
    lu_op = Number.operation_counter
    
    Number.counter_reset()
    t_start = time()
    det(A)
    det_time = time() - t_start
    det_op = Number.operation_counter
    
    res_times.append((inverse_time, lu_time, det_time))
    res_operations.append((inverse_op, lu_op, det_op))


In [13]:
res_times = np.array(res_times)
print([x[0] for x in res_times])
print([x[1] for x in res_times])
print([x[2] for x in res_times])

[0.0, 0.0010004043579101562, 0.001001119613647461, 0.006001949310302734, 0.04471921920776367, 0.36334824562072754, 2.9337551593780518]
[0.0, 0.0, 0.00099945068359375, 0.0070002079010009766, 0.03673744201660156, 0.3025834560394287, 2.236036539077759]
[0.0, 0.0, 0.0009996891021728516, 0.0048828125, 0.03705596923828125, 0.3551802635192871, 2.287510871887207]


In [23]:
res_operations = np.array(res_operations)
print([x[0] for x in res_operations])
print([x[1] for x in res_operations])
print([x[2] for x in res_operations])

[3, 134, 1420, 12568, 105008, 857184, 6924480]
[8, 86, 944, 9200, 81032, 677096, 5525512]
[12, 94, 960, 9232, 81096, 677224, 5525768]


In [29]:

df = pd.DataFrame(index = ["Inverse", "LU", "det"], columns = sizes)
df.loc['Inverse'] = res_times[:, 0]
df.loc['LU'] = res_times[:, 1]
df.loc['det'] = res_times[:, 2]
print("------------------------------------Times------------------------------------")
print(df)

------------------------------------Times------------------------------------
         2      4         8         16        32        64        128
Inverse  0.0  0.001  0.001001  0.006002  0.044719  0.363348  2.933755
LU       0.0    0.0  0.000999     0.007  0.036737  0.302583  2.236037
det      0.0    0.0     0.001  0.004883  0.037056   0.35518  2.287511


In [31]:
df1 = pd.DataFrame(index = ["Inverse", "LU", "det"], columns = sizes)
df1.loc['Inverse'] = res_operations[:, 0]
df1.loc['LU'] = res_operations[:, 1]
df1.loc['det'] = res_operations[:, 2]
print("--------------------------Floating point operations--------------------------")
print(df1)

--------------------------Floating point operations--------------------------
        2    4     8      16      32      64       128
Inverse   3  134  1420  12568  105008  857184  6924480
LU        8   86   944   9200   81032  677096  5525512
det      12   94   960   9232   81096  677224  5525768


Wykresy

In [ ]:
plt.plot(sizes, res_times)
plt.title("times comparision")
plt.xlabel("size")
plt.ylabel("time [s]")
plt.xticks(sizes)
plt.legend(("inverse", "lu_factorization", "determinant"))
plt.show()

In [ ]:
plt.plot(sizes, res_operations)
plt.title("no_operations comparision")
plt.xlabel("size")
plt.ylabel("number of operations")
plt.xticks(sizes)
plt.legend(("inverse", "lu_factorization", "determinant"))
plt.show()

# Złożoność

In [32]:
from scipy.optimize import curve_fit

In [33]:
def func(x, a, b):
    return a * np.power(x,b)

In [40]:
x_data = sizes
y_det_op = res_operations[:, 2]
y_lu_op = res_operations[:, 1]
y_inv_op = res_operations[:, 0]

y_det_time = res_times[:, 2]
y_lu_time = res_times[:, 1]
y_inv_time = res_times[:, 0]

In [47]:
params_inv_op, cov_inv_op = curve_fit(func, x_data, y_inv_op)
print(params_inv_op, "\n",  cov_inv_op)

[3.07741239 3.01450861] 
 [[ 1.80399444e-04 -1.21046245e-05]
 [-1.21046245e-05  8.12471307e-07]]


In [48]:
params_inv_time, cov_inv_time = curve_fit(func, x_data, y_inv_time)
print(params_inv_time, "\n",  cov_inv_time)

[1.31023574e-06 3.01349882e+00] 
 [[ 1.44793968e-16 -2.28193894e-11]
 [-2.28193894e-11  3.59747975e-06]]


------------------------------------------------------------------------------------------------

In [49]:
params_lu_op, cov_lu_op = curve_fit(func, x_data, y_lu_op)
print(params_lu_op, "\n",  cov_lu_op)

[2.28086866 3.02972878] 
 [[ 4.80811955e-04 -4.35270603e-05]
 [-4.35270603e-05  3.94167638e-06]]


In [50]:
params_lu_time, cov_lu_time = curve_fit(func, x_data, y_lu_time)
print(params_lu_time, "\n",  cov_lu_time)

[1.81601510e-06 2.89025392e+00] 
 [[ 6.31604280e-15 -7.18430949e-10]
 [-7.18430949e-10  8.17510111e-05]]


-------------------------------------------

In [51]:
params_det_op, cov_det_op = curve_fit(func, x_data, y_det_op)
print(params_det_op, "\n", cov_det_op)

[2.2834843  3.02950209] 
 [[ 4.61316591e-04 -4.17143681e-05]
 [-4.17143681e-05  3.77320028e-06]]


In [52]:
params_det_time, cov_det_time = curve_fit(func, x_data, y_det_time)
print(params_det_time, "\n",  cov_det_time)

[4.47765654e-06 2.70898018e+00] 
 [[ 4.60195563e-13 -2.12440762e-08]
 [-2.12440762e-08  9.81187649e-04]]
